## Analysis
This project demonstrated the usage of pkdb for the test substance caffeine.
### Data Sources
- pkdb rest api : "http://0.0.0.0:8000/api/v1/"

### Outputs:
- data/processed/outputs.tsv
- data/processed/timecourses.tsv
- data/processed/interventions.tsv
- data/processed/individuals.tsv
- data/processed/groups.tsv

#### Groups and Individuals 
- data/processed/all_subjects.tsv

### Merged Dataframes
- data/processed/all_subjects.tsv

### Normalized Dataframes
- data/processed/all_subjects.tsv

### Changes
- 11-29-2018 : Started project

In [32]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
from utils import PkdbModel , Preprocessed
import pandas as pd


In [34]:
queries = ['outputs','timecourses','interventions','individuals','groups',"studies","substances"]
#queries = ["studies","substances"]
#queries = ["groups"]

In [35]:
pk_instance = PkdbModel("outputs")  


### Query, Process and Save
Json data is loaded, transformed to dataframes, preprocessed and saved as tab seperated files.

In [36]:
for pk_instance in queries:
    pk_instance = PkdbModel(pk_instance)  
    pk_instance.load()
    pk_instance.preprocess()
    pk_instance.save()
    pk_instance.report()

____________________________________________________________
Name: outputs
Loaded: True
Preprocessed: True
saved: True
outputs were succsesfully saved to </home/janek/Dev/pkdb_analysis/data/1-preprocessed/outputs.tsv>
____________________________________________________________
Name: timecourses
Loaded: True
Preprocessed: True
saved: True
timecourses were succsesfully saved to </home/janek/Dev/pkdb_analysis/data/1-preprocessed/timecourses.tsv>
____________________________________________________________
Name: interventions
Loaded: True
Preprocessed: True
saved: True
interventions were succsesfully saved to </home/janek/Dev/pkdb_analysis/data/1-preprocessed/interventions.tsv>
____________________________________________________________
Name: individuals
Loaded: True
Preprocessed: True
saved: True
individuals were succsesfully saved to </home/janek/Dev/pkdb_analysis/data/1-preprocessed/individuals.tsv>
____________________________________________________________
Name: groups
Loaded: True

### Convert Units


In [37]:
this_data = PkdbModel('outputs',destination='1-preprocessed')
this_data.read()
this_data.data.groupby(["calculated"]).apply(len)

calculated
False    5796
True     4169
dtype: int64

In [38]:
this_data = PkdbModel('interventions',destination='1-preprocessed')
this_data.read()
this_data.data.groupby(["unit"]).apply(len)

unit
IU / hour / kilogram        1
day                         1
gram                      255
gram / hour                 3
gram / kilogram            36
mole                        1
mole / hour / kilogram      4
mole / kilogram             1
dtype: int64

In [39]:
test = this_data.data[this_data.data["unit"] == "microgram * milliliter / hour / micromole"]

In [40]:
test.groupby("substance_name").apply(len)

""
substance_name


In [41]:
this_data = PkdbModel('outputs',destination='1-preprocessed')
this_data.read()
this_data.data.groupby(["pktype","unit"]).count()

interventions  \
pktype                 unit                                                       
HOMA-IR                none                                                   6   
Insulinogenic index    none                                                   6   
Matsuda index          none                                                   6   
Oral disposition index none                                                   6   
QUICKI                 none                                                   6   
auc_end                IU * hour / liter                                     11   
                       gram * hour / liter                                  661   
                       hour * mole / liter                                   14   
auc_inf                IU * hour / liter                                     29   
                       gram * hour / liter                                  844   
                       hour * mole / liter                                   15   
auc_relative           none                                                  27   
aumc_inf               gram * hour ** 2 / liter                               2   
clearance              gram * milliliter / IU / hour                          7   
                       gram * milliliter / IU / hour / kilogram               7   
                       liter / hour                                         776   
                       liter / hour / kilogram                              698   
                       microgram * milliliter / hour / micromole             14   
                       milliliter / hour / micromole                          4   
                       milliliter / meter ** 2 / minute                       2   
clearance_intrinsic    liter / hour                                           2   
clearance_partial      liter / hour / kilogram                               51   
clearance_renal        liter / hour                                          72   
                       liter / hour / kilogram                               20   
clearance_unbound      liter / hour                                           2   
                       liter / hour / kilogram                                9   
cmax                   IU / liter                                            11   
                       gram / liter                                         876   
                       mole / liter                                          14   
concentration          gram / liter                                         149   
cum_amount             gram                                                   2   
fraction_absorbed      none                                                   8   
fraction_unbound       none                                                   4   
kabs                   1 / minute                                             5   
kel                    1 / minute                                           711   
mrt                    hour                                                  40   
oro-cecal transit time hour                                                  12   
plasma_binding         none                                                   5   
ratio                  none                                                1232   
recovery               gram                                                   4   
                       none                                                 336   
thalf                  hour                                                1311   
thalf_absorption       hour                                                   1   
tmax                   hour                                                 970   
vd                     liter                                                509   
                       liter * milligram / microIU                            7   
                       liter * milligram / millimole                         14   

### Merge
 - merge individuals and groups  --> all_subjects,
 - merge outputs and timecourses --> all_results, 
 - merge all_results with interventions and individuals  --> individuals_complete, 
 - merge all_results with interventions and groups  --> groups_complete, 
 - merge all_results with interventions and all_subjects -->  all_complete

In [42]:
prepocessed = Preprocessed()

In [43]:
prepocessed.read()
prepocessed.merge()
prepocessed.save()